# Neo4j와 LangChain을 활용한 영화 추천 시스템

---

## 1. Neo4J DB 환경 설정

In [ ]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv(override=True)

In [ ]:
from langchain_neo4j import Neo4jGraph

# LangChain 도구 활용 - DB 연결 객체 초기화 
graph = Neo4jGraph( 
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
    enhanced_schema=True, # 확장 스키마 출력 설정
)

In [ ]:
# 테스트 쿼리 실행 
cypher_query = """
MATCH (n:Movie)
RETURN COUNT(n) AS Movie_Count
"""

graph.query(cypher_query)

---

## 2. Text-to-Cypher 활용

Text-to-Cypher는 자연어 질의를 Neo4j 데이터베이스 쿼리 언어인 Cypher로 변환하여 지식 그래프를 효과적으로 검색할 수 있는 기술입니다. 영화 데이터베이스에서 사용자 질문을 Neo4j Cypher 쿼리로 변환하는 방법을 배우게 됩니다. 

- **학습 목표**:

   - LLM을 활용하여 자연어 질의를 Cypher 쿼리로 변환하는 방법 이해
   - `GraphCypherQAChain`의 사용법 익히기
   - 다양한 영화 데이터 쿼리 패턴 학습
   - 커스텀 프롬프트를 활용한 정교한 쿼리 생성 방법 습득

- **필요 사항**:
   - Neo4j 데이터베이스 (영화 데이터 포함)
   - 필요 패키지: `langchain-neo4j`, `langchain-openai`

### 2.1 영화 데이터베이스 스키마 확인

영화 데이터베이스는 다음과 같은 노드와 관계로 구성되어 있습니다:

- **노드 속성:**
   - **Person {id: STRING, name: STRING}**: 영화 관련 인물 정보
   - **Movie {id: STRING, title: STRING, released: STRING, rating: FLOAT, overview: STRING, runtime: INTEGER, tagline: STRING, content_embedding: LIST}**: 영화 정보
   - **Genre {id: STRING, name: STRING}**: 영화 장르 정보

- **관계:**
   - **(:Person)-[:ACTED_IN]->(:Movie)**: 배우와 영화 간의 출연 관계
   - **(:Person)-[:DIRECTED]->(:Movie)**: 감독과 영화 간의 감독 관계
   - **(:Movie)-[:IN_GENRE]->(:Genre)**: 영화와 장르 간의 분류 관계

In [ ]:
graph.refresh_schema()
print(graph.schema)

### 2.2 LangChain을 활용한 Text-to-Cypher 구현

- LangChain으로 Neo4J 지식 그래프 조회
- LLM을 사용하여 사용자 질의를 Cypher 쿼리로 변환

#### 1) **GraphCypherQAChain** 설정

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_neo4j import GraphCypherQAChain

# LLM 및 그래프 객체 초기화
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.0)

# GraphCypherQAChain 객체 초기화
# 자연어 질의를 Cypher 쿼리로 변환하고 실행하는 체인
# llm: 질의를 Cypher로 변환할 LLM 모델
# graph: 쿼리를 실행할 Neo4j 그래프 객체
# allow_dangerous_requests=True: 잠재적으로 위험한 쿼리도 허용
# verbose=True: 중간 과정과 디버깅 정보를 상세히 출력
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm, 
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True,
)

#### 2) **기본 영화 쿼리** 

- Text to Cypher (DB 조회)

In [ ]:
# 영화 제목으로 정보 검색
answer = cypher_chain.invoke({"query": "영화 'Apollo 13'에 대한 정보를 알려주세요."})

In [ ]:
# 답변 출력
print(answer)

In [ ]:
# LLM 답변 출력
from pprint import pprint
pprint(answer['result'])

In [ ]:
# 특정 감독의 영화 찾기
answer = cypher_chain.invoke({"query": "'Christopher Nolan' 감독의 영화를 모두 찾아주세요."})

In [ ]:
# 답변 출력
print(answer)

In [ ]:
# LLM 답변 출력
pprint(answer['result'])

In [ ]:
# 특정 배우가 출연한 영화 찾기
answer = cypher_chain.invoke({"query": "'Tom Hanks'가 출연한, 평점이 가장 높은 영화 3개는 무엇인가요?"})

In [ ]:
# 답변 출력
print(answer)

In [ ]:
# LLM 답변 출력
pprint(answer['result'])

In [ ]:
# 장르별 영화 검색
answer = cypher_chain.invoke({"query": "2010년 이후 개봉한 'Drama' 장르 영화 중 평점이 높은 순서로 5개를 보여주세요."})

In [ ]:
# 답변 출력
print(answer)

In [ ]:
# LLM 답변 출력
pprint(answer['result'])

#### 3) **출력 갯수를 지정 (top k)** 

In [ ]:
# top_k 파라미터로 결과 수 제한
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True,
    top_k=5,  # 최대 5개의 결과만 반환
)

answer = cypher_chain.invoke({"query": "'Tom Hanks' 배우가 출연한 영화를 모두 알려주세요."})

In [ ]:
# LLM 답변 출력
pprint(answer['result'])

#### 4) **중간 과정 확인** 

- `return_intermediate_steps`=True

In [ ]:
# 중간 결과를 포함하여 출력
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True,
    return_intermediate_steps=True  # 생성된 Cypher 쿼리와 중간 결과 확인
)

# 평점 8점 이상인 액션 영화 찾기
answer = cypher_chain.invoke({"query": "평점이 8점 이상인 'Action' 장르 영화는 무엇이 있나요?"})

In [ ]:
# 중간 결과 확인
for k, v in answer.items():
    print(f"{k}: {v}")

#### 5) **직접 Cypher 결과 얻기 (LLM 답변 없이)** 

- `return_direct`=True

In [ ]:
# 직접 Cypher 결과 얻기 (LLM 답변 없이)
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True,
    return_direct=True  # LLM 답변 생성 단계 건너뛰기
)

# 2000년대 개봉한 영화 중 평점 순 정렬
answer = cypher_chain.invoke({"query": "2000년대(2000-01-01 ~ 2009-12-31) 개봉한 영화를 평점 순으로 정렬해주세요."})

In [ ]:
for k, v in answer.items():
    print(f"{k}: {v}")

In [ ]:
import pandas as pd
pd.DataFrame([item['m'] for item in answer['result']])

#### 6) **다양한 LLM 모델 활용** 

- cypher 쿼리 생성하는 모델과 최종 답변 생성 모델을 다르게 적용

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Cypher 쿼리 생성은 OpenAI 모델, 최종 답변은 Gemini 모델 사용
cypher_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.0)
qa_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0)

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=cypher_llm,  # Cypher 쿼리 생성용 LLM
    qa_llm=qa_llm,          # 최종 답변 생성용 LLM
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True,
)

# 특정 배우와 감독이 함께 작업한 영화 찾기
answer = cypher_chain.invoke({"query": "배우 'Leonardo DiCaprio'와 감독 'Christopher Nolan'이 함께 작업한 영화가 있나요?"})

In [ ]:
for k, v in answer.items():
    print(f"{k}: {v}")

#### 7) **커스텀 프롬프트 활용**

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate

# Cypher 생성을 위한 영화 데이터베이스 특화 프롬프트
CYPHER_GENERATION_TEMPLATE = """Task: Generate Cypher statement to question a movie graph database.
Instructions:
- Use only the provided node labels, relationship types, and properties in the schema.
- Do not use any relationship types or properties not specified in the schema.
- Focus on extracting meaningful insights from movie data.

Schema:
{schema}

Note: 
- Provide only the Cypher statement.
- Do not include explanations or apologies.
- Generate precise, relevant Cypher queries.

Examples:
# 특정 배우가 출연한 영화 찾기
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
RETURN m.title, m.released, m.rating
ORDER BY m.released DESC

# 특정 장르의 평점 높은 영화 찾기
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)
WHERE g.name = 'Action'
RETURN m.title, m.rating
ORDER BY m.rating DESC
LIMIT 10

# 특정 감독의 영화 중 가장 흥행한 작품
MATCH (p:Person)-[:DIRECTED]->(m:Movie)
WHERE p.name = 'Steven Spielberg'
RETURN m.title, m.rating
ORDER BY m.rating DESC
LIMIT 5

# 특정 연도에 개봉한 영화 조회
MATCH (m:Movie)
WHERE m.released = 2000
RETURN m.title, m.rating
ORDER BY m.rating DESC

# 특정 배우와 감독이 함께 작업한 영화 찾기
MATCH (actor:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(director:Person)
WHERE actor.name = 'Leonardo DiCaprio' AND director.name = 'Christopher Nolan'
RETURN m.title, m.released, m.rating

The question is:
{question}"""

# 결과 처리를 위한 영화 QA 프롬프트
QA_TEMPLATE = """
당신은 영화 데이터베이스 분석 전문가로, 영화 정보에 대한 명확하고 간결한 정보를 한국어로 제공합니다.

[질문]
{question}

[검색 결과]
{context}

# 응답 가이드라인:
- 검색 결과에서 핵심 정보를 요약하세요
- 영화 데이터에 대한 명확하고 객관적인 개요를 제공하세요
- 전문적이고 유익한 톤을 사용하세요
- 영화 데이터에서 중요한 패턴이나 트렌드를 강조하세요
- 맥락이 불충분한 경우 더 많은 정보가 필요하다고 명확히 언급하세요
- 추측이나 개인적인 해석은 피하세요

# 응답 형식:
- 간략한 발견 요약으로 시작하세요
- 여러 영화가 발견된 경우 간결한 개요를 제공하세요
- 가독성을 위해 글머리 기호나 짧은 단락을 사용하세요
- 개봉일, 평점, 주요 배우나 감독과 같은 관련 세부 정보를 포함하세요
- 모든 숫자 데이터나 기술 용어를 이해하기 쉬운 언어로 번역하세요

# 예시 응답 구조:
"분석 결과, [주요 발견 요약]

주요 특징:
- [첫 번째 중요 인사이트]
- [두 번째 중요 인사이트]

추가 정보: [필요한 경우 추가 설명]"
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

QA_PROMPT = PromptTemplate(
    input_variables=["question", "context"], 
    template=QA_TEMPLATE
)

# Chain 생성 - 주목: input_key와 output_key를 명시적으로 설정
cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=cypher_llm,
    qa_llm=qa_llm,
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    qa_prompt=QA_PROMPT,
    input_key="question",  
    output_key="result"
)

# Cypher 쿼리 실행
answer = cypher_chain.invoke({"question": "배우 'Leonardo DiCaprio'와 감독 'Christopher Nolan'이 함께 작업한 영화가 있나요?"})

In [ ]:
for k, v in answer.items():
    print(f"{k}: {v}")